# Multimodal Accessibility Analysis *At Scale*: 
## Measuring Access to Covid Testing Sites in California

This notebook calculates access to Covid-19 testing sites for every block group in California. Apart from the locations of test sites, all data are collected on the fly. To account for the massive size of the study area (all of CA) the analysis is chunked at the county level; access is computed for the blockgroups in each county (with both supply and demand data buffered 8km beyond the county borders to mitigate edge effects), then the counties are recombined into a statewide set. 

In [1]:
import cenpy
import osmnet
import os
import importlib
import pandas as pd
import numpy as np
import pandana as pdna
import urbanaccess as ua
import geopandas as gpd
import matplotlib.pyplot as plt
from geosnap import Community, datasets
from access import access as Access

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/_data.py:123: UserWarning: Unable to locate local census data. Streaming instead.
If you plan to use census data repeatedly you can store it locally with the io.store_census function for better performance
  "Unable to locate local census data. Streaming instead.\n"
Loading manifest: 100%|██████████| 5/5 [00:00<00:00, 11683.30entries/s]


In [2]:
from access import weights as acweights
from healthacc.travel_matrix import compute_travel_cost_adjlist

In [3]:
from segregation.util import project_gdf

## Testing Locations (testing supply)

In [4]:
test_locations = gpd.read_file("../data/accessbility/Testing_Locations_7_15.shp")

In [5]:
test_locations['count'] = 1

## Census Data (testing demand)

In [6]:
counties = datasets.counties()

In [7]:
ca_counties = counties[counties.geoid.str.startswith('06')]

In [8]:
ca_blkgrps= gpd.read_file("../data/accessbility/CA_BG_Income.shp")

In [9]:
ca_blkgrps = ca_blkgrps.to_crs(4326)

In [10]:
counties = [county[-3:] for county in ca_counties.geoid.tolist()]

Our blockgroup geometries seem to contain income data, but we also need population data to pass to `access`. We'll grab blockgroup-level population data using `cenpy`, but since blockgroups [arent implemented]() in the products api yet, we need to loop over each county

In [11]:
if not os.path.exists('../data/ca_blockgroup_population.csv'):

    cadict = []
    conn = cenpy.products.APIConnection("ACSDT5Y2017")   
    for county in counties:
        data = conn.query(["B01001_001E"], geo_unit = 'block group', geo_filter = {"state": "06", "county": f'{county}'})
        cadict.append(data)
    gdf = pd.concat(cadict)
    gdf.to_csv('../data/ca_blockgroup_population.csv')
else:
    gdf = pd.read_csv('../data/ca_blockgroup_population.csv', converters={'state':str, 'county':str, 'tract':str, 'block group': str})

In [12]:
gdf['geoid'] = gdf.state + gdf.county + gdf.tract + gdf['block group']

In [13]:
blockgroups = ca_blkgrps.merge(gdf, left_on='GEOID10', right_on='geoid')

## OSM Data (pedestrian network)

To download the networks for each county we need to do a little bit of laborious data processing. The study area of *all* of california is too large of a problem size to deal with in a single go (we're likely talking  about a point-to-point travel matrix of billions of OSM intersections...) so we split the data and process by county. To mitigate edge effects though, we need to buffer each county beyond the travel threshold we care about and operate on the buffered data before clipping it back to the proper spatial extent. OSM handles data in lat/long, so we first need to iterate over each california county and project it into the proper CRS. California is a big state, so it falls into two different UTM zones (10 and 11). I don't want to lookup the proper CRS for each county, so here we use the `utm` library to:
- loop over each county
- project it to utm
- buffer it out 8000m
- project it back to WGS and collect the bounding box
- send the bbox to OSM to download the network and store it

In [14]:
def determine_utm(polygon):
    
    lon = polygon.centroid.x
    lat = polygon.centroid.y
    
    zone = utm.from_latlon(lat,lon)
    crs = crs = f"+proj=utm +zone={str(zone[2])} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    return crs

In [15]:
def buffer_bbox(polygon):
    
    crs = determine_utm(polygon)
    gdf = gpd.GeoDataFrame(pd.Series([polygon], name='geometry'))
    gdf.crs=4326
    gdf = gdf.to_crs(crs).buffer(8000)
    gdf = gdf.to_crs(4326)
    
    return gdf.total_bounds
    

In [16]:
for county in ca_counties.iterrows():
    fname = f"../data/networks/osmnet_{county[1].geoid}.h5"
    if not os.path.exists(fname):
        bbox = buffer_bbox(county[1].geometry)
        nodes, edges = osmnet.network_from_bbox(bbox=tuple(bbox))
        net = pdna.Network(nodes["x"],
                           nodes["y"],
                           edges["from"],
                           edges["to"],
                           edges[["distance"]])
        net.save_hdf5(fname)
    else:
        pass
    

## GTFS Data (transit network)

In my first attempt, I used the builtin downloader but the results were unsatisfactory, so I wrote a couple functions to hit the transitland api instead, which works better

~~Brute force: add all feeds for california*, then clip the network to an extent using the buffered bboox for each county. GTFS data are pretty small so this is easy and fast~~

~~**two things to note here. First, urbanaccess is setup to query from GTFS Data Exchange which was shut down a few years ago, so the data may be stale. Second, we're doing a string search for California, which may not necessarily be in every CA transit feed name. TransitLand has popped up to replace  GTGS Data Exchange, and its api allows searching by geograpic extent via bounding boxes. There's an experimental (but stale) branch of ua that implements a transitland search here https://github.com/UDST/urbanaccess/blob/kuanb-gtfs-feeds-query-util/urbanaccess/gtfsfeeds.py but i havent tested it. ~~

In [17]:
from healthacc import feeds_from_bbox

In [18]:
# these throw a 403. I think the call to `requests` inside urbanaccess needs to be beefed up with user agent and headers and such 
# <https://stackoverflow.com/questions/38489386/python-requests-403-forbidden>
fourofours = ['metro', 'laketahoe','southwestpoint','metrolinktrains', 'riversidetransitagency','blue','tidelinewatertaxi','countyconnection']

# throw  401 for not having an API key for 511.org
fouroones = ['capitolcorridor','angelislandtiburonferry', 'acealtamontcorridorexpress', 'goldengateferry', 'alcatrazhornblowerferry', 'smart', \
            'unioncitytransit', 'vacavillecitycoach', 'vinenapacounty', 'sanfranciscobayferry', 'americancanyontransit', 'dumbartonexpress']
#this one times out
tout = ['trideltatransit']
# these are corrupt zips or bad GTFS data
# usually missing calendar_dates, but other errors too
incorrect = ['commuteorgshuttle', 'culvercitybus', 'airportexpressinc', 'madera','anaheim','vctc','goldengatetransit','kerncounty','delnorte',\
            'yosemite','longbeachtransit', 'getbus', 'avalon', 'amtrak', 'petalumatransit'] 

feederrors = fourofours + fouroones + tout + incorrect




In [77]:
def build_network(county, bbox):
    COUNTY_PATH = f"../data/counties/{county}"
    osm_network = pdna.Network.from_hdf5(f"../data/networks/osmnet_{county}.h5")
    try:
        # download gtfs data if we need it
        if os.path.exists(f"{COUNTY_PATH}/{county}_gtfs.h5"):
            loaded_feeds = ua.gtfs.network.load_processed_gtfs_data(f"{county}_gtfs.h5", dir=COUNTY_PATH,)
        else:   
            # get GTFS feeds for the area
            feeds = feeds_from_bbox(bbox)
            for feed in list(feeds.keys()):
                if feed in feederrors:
                    feeds.pop(feed)
            if len(ua.gtfsfeeds.feeds.to_dict()['gtfs_feeds'])>0:
                ua.gtfsfeeds.feeds.remove_feed(remove_all=True)  #feeds object is global so reset it from last iter
            ua.gtfsfeeds.feeds.add_feed(feeds)
            ua.gtfsfeeds.download()
            loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(f"{COUNTY_PATH}/gtfsfeed_text/",
                                                       bbox=bbox,
                                                       remove_stops_outsidebbox=True)
            ua_to_h5(loaded_feeds, f"{COUNTY_PATH}/{county}_gtfs.h5")
        
        # Create transit and OSM networks and combine them into a single multimodal net
        ua.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                       day='monday',
                                       timerange=['07:00:00', '10:00:00'],
                                       calendar_dates_lookup=None)
        osm_network.nodes_df['id'] = osm_network.nodes_df.index
        ua_osm = ua.create_osm_net(osm_edges=osm_network.edges_df,
                                  osm_nodes=osm_network.nodes_df,
                                  travel_speed_mph=3)
        urbanaccess_net = ua.ua_network
        ua.integrate_network(urbanaccess_network=urbanaccess_net,
                             headways=False)
        combined_net = pdna.Network(urbanaccess_net.net_nodes["x"],
                                    urbanaccess_net.net_nodes["y"],
                                    urbanaccess_net.net_edges["from_int"],
                                    urbanaccess_net.net_edges["to_int"],
                                    urbanaccess_net.net_edges[["weight"]])
    
    except KeyError:  # fallback to ped-only network if no gtfs in that county
        combined_net = pdna.Network(osm_network.nodes_df["x"],
                                    osm_network.nodes_df["y"],
                                    osm_network.edges_df["from"],
                                    osm_network.edges_df["to"],
                                    osm_network.edges_df[["weight"]])
    return combined_net

In [20]:
def ua_to_h5(loaded_feeds, path):
    
    hdf = pd.HDFStore(path)
    hdf['calendar'] =loaded_feeds.calendar
    hdf['calendar_dates'] =loaded_feeds.calendar_dates
    hdf['headways'] =loaded_feeds.headways
    hdf['routes'] =loaded_feeds.routes
    hdf['stop_times'] =loaded_feeds.stop_times
    hdf['stop_times_int'] =loaded_feeds.stops
    hdf['stops'] =loaded_feeds.stops
    hdf['trips'] = loaded_feeds.trips
    hdf.close()

## Calculating Accessibility

In [21]:
def prepare_data(county):
    
    df = ca_blkgrps[ca_blkgrps.GEOID10.str[:5]==county].copy()
    boundary = project_gdf(df).buffer(8000)
    utm = boundary.crs
    boundary = boundary.to_crs(4326).unary_union
    
    # this part is expensive
    buf_testint = test_locations[test_locations.intersects(boundary)]    
    buf_blkgrps = ca_blkgrps[ca_blkgrps.centroid.intersects(boundary)]

    geoms = buf_blkgrps[['GEOID10', 'geometry']]
    
    
    dests = gpd.sjoin(geoms, buf_testint, how='left', op='intersects').groupby('GEOID10').sum().merge(geoms, on='GEOID10')
    
    buf_blkgrps = buf_blkgrps.merge(dests[['GEOID10', 'count']], on='GEOID10')
    buf_blkgrps = buf_blkgrps.merge(blockgroups[['GEOID10', 'B01001_001E']], on='GEOID10', how='left')
    buf_blkgrps =  gpd.GeoDataFrame(buf_blkgrps)
    buf_blkgrps.crs = 4326

    
    return buf_blkgrps



In [76]:
def compute_access(county, blkgrps, network):
    COUNTY_PATH = f"../data/counties/{county}"
    if os.path.exists(f"{COUNTY_PATH}/matrices/{county}_adj.parquet"):
        adjlist = pd.read_parquet(f"{COUNTY_PATH}/matrices/{county}_adj.parquet")
    else:
        adjlist = compute_travel_cost_adjlist(blkgrps, blkgrps, network, reindex_dest='GEOID10', reindex_orig='GEOID10')
        adjlist.assign(temp=adjlist.origin+adjlist.destination).drop_duplicates(subset='temp', inplace=True)
    ac_test=Access(demand_df = blkgrps,
            demand_index = 'GEOID10',
            demand_value = 'B01001_001E',
            supply_df    = blkgrps,
            supply_index = 'GEOID10',
            cost_df=adjlist.replace(0.0,1.0),  # gravity chokes if travel time is 0
            cost_origin='origin',
            cost_dest='destination',
            cost_name='cost',
            supply_value='count',
            neighbor_cost_df     = adjlist.replace(0.0,1.0),
            neighbor_cost_origin = 'origin',
            neighbor_cost_dest   = 'destination',
            neighbor_cost_name   = 'cost'
      )
    ac_test.raam(name = "raam", tau = 60);
    ac_test.access_df["raam_count"] = 1 / ac_test.access_df["raam_count"] 
    ac_test.two_stage_fca(name ="2sfca", max_cost = 60,)
    ac_test.enhanced_two_stage_fca(name = "g2sfca", weight_fn = gaussian)
    ac_test.three_stage_fca(name = "3sfca")
    ac_test.weighted_catchment(name = "catch_gravity", weight_fn = gravity)
    ac_test.weighted_catchment(name = "catch_gaussian", weight_fn = gaussian)
    ac_test.fca_ratio(name = "fca60",      max_cost = 30)
    ac_test.fca_ratio(name = "fca120",      max_cost = 60) 
    
    return ac_test.access_df[ac_test.access_df.index.str.startswith(county)]


In [27]:
from urbanaccess.config import settings

In [28]:
settings.to_dict()

{'data_folder': 'data',
 'logs_folder': 'logs',
 'log_file': True,
 'log_console': False,
 'log_name': 'urbanaccess',
 'log_filename': 'urbanaccess',
 'gtfs_api': {'gtfsdataexch': 'http://www.gtfs-data-exchange.com/api/agencies?format=csv'}}

In [29]:
len(ua.gtfsfeeds.feeds.to_dict()['gtfs_feeds'])

0

In [30]:
from access import weights as acweights

In [31]:
gravity = acweights.gravity(scale = 60, alpha = -1)
gaussian = acweights.gaussian(60)

In [73]:
settings.log_console=False # turn off urbanccess verbosity

This cell does all the computation and it will take awhile to run. Despite pandana's impressive speed, this is a lot of data to crunch. Most counties are done in a minute or two, but LA takes about an hour on its own.

In [ ]:
ca_statewide = []
for index, row in ca_counties.iterrows():
    try:
        if not os.path.exists(f"../data/counties/{row.geoid}_access.parquet"):

            gdf = prepare_data(row.geoid)
            network = build_network(row.geoid, gdf.total_bounds)
            access = compute_access(row.geoid, gdf.drop_duplicates(subset=['GEOID10']), network)
            access.to_parquet(f"../data/counties/{row.geoid}_access.parquet")
        else:
            access = pd.read_parquet(f"../data/counties/{row.geoid}_access.parquet")
        ca_statewide.append(access)
    except:
        print(f"{row.geoid} failed")
statewide_access = pd.concat(ca_statewide)
statewide_access.to_parquet("../data/ca_statewide_access.parquet")

/Users/knaaptime/anaconda3/envs/healthacc/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  # Remove the CWD from sys.path while we load stuff.


Successfully read store: ../data/counties/06037/06037_gtfs.h5 with the following keys: ['/calendar', '/calendar_dates', '/headways', '/routes', '/stop_times', '/stop_times_int', '/stops', '/trips']
Successfully read store: ../data/counties/06037/06037_gtfs.h5 with the following keys: ['/calendar', '/calendar_dates', '/headways', '/routes', '/stop_times', '/stop_times_int', '/stops', '/trips']
Successfully read store: ../data/counties/06037/06037_gtfs.h5 with the following keys: ['/calendar', '/calendar_dates', '/headways', '/routes', '/stop_times', '/stop_times_int', '/stops', '/trips']
Successfully read store: ../data/counties/06037/06037_gtfs.h5 with the following keys: ['/calendar', '/calendar_dates', '/headways', '/routes', '/stop_times', '/stop_times_int', '/stops', '/trips']
Successfully read store: ../data/counties/06037/06037_gtfs.h5 with the following keys: ['/calendar', '/calendar_dates', '/headways', '/routes', '/stop_times', '/stop_times_int', '/stops', '/trips']
Successful

In [44]:
statewide_access

,B01001_001E,raam_count,2sfca_count,g2sfca_count,3sfca_count,catch_gravity_count,catch_gaussian_count,fca60_count,fca120_count
GEOID10,,,,,,,,,
060910100001,822.0,0.067329,0.000000,4.666358e-45,0.000000,0.072190,2.146525e-42,0.000000,0.000000
060910100002,755.0,0.075626,0.000000,7.855203e-36,0.000000,0.081813,3.613393e-33,0.000000,0.000000
060910100003,460.0,0.983607,0.002174,2.173611e-03,0.002091,60.000000,9.998611e-01,0.002174,0.002174
060910100004,496.0,0.149394,0.000000,1.984793e-10,0.000000,0.175633,9.130048e-08,0.000000,0.000000
060910100005,352.0,0.102245,0.000000,3.945663e-20,0.000000,0.113890,1.815005e-17,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
060790128001,200.0,0.611836,0.000696,7.278385e-04,0.000001,5.243206,1.320800e+00,0.000000,0.000078
060790129001,2233.0,0.314930,0.000000,7.369267e-05,0.000000,3.314752,1.195730e-01,0.000000,0.000000
060790129002,2745.0,0.286054,0.000000,3.594200e-04,0.000000,3.357944,2.833098e-01,0.000000,0.000000


In [ ]:
blockgroups = blockgroups.merge(statewide_access, left_on='GEOID10', right_index=True)

In [ ]:
blockgroups.plot('raam_count', cmap='blues', scheme='quantiles', k=8)